In [84]:
import requests
import datetime
from transformers import pipeline
from bs4 import BeautifulSoup
from config import config
from pyBM25 import BM25
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import re

In [2]:
cfig = config()
polykey = cfig.polygon

In [83]:
stopwords=[]
with open('stopwords.txt') as f:
    stopwords.append(f.read().splitlines())

In [7]:
bm25 = BM25(stopwords=stop_words)

NameError: name 'stop_words' is not defined

In [146]:
ticker = "CMG"
published = "11/01/2021"
r_lim = "100"

In [147]:
#Polygon ticker information request
url = ('https://api.polygon.io/v1/meta/symbols/'
      +ticker+'/'
      +'company?'
      'apikey='+polykey
      )

In [148]:
response = requests.get(url)
if response.status_code==200:
    response = response.json()
else:
    print(response.status_code)

In [149]:
#Show available items
for k,v in response.items():
    print(k)

logo
listdate
cik
bloomberg
figi
lei
sic
country
industry
sector
marketcap
employees
phone
ceo
url
description
exchange
name
symbol
exchangeSymbol
hq_address
hq_state
hq_country
type
updated
tags
similar
active


In [150]:
print(response['country'])

usa


In [151]:
print(response['tags'])

['Consumer Cyclical', 'Restaurants']


In [152]:
print(response['similar'])

['PZZA', 'JACK', 'DENN', 'BJRI', 'MCD', 'YUM', 'DPZ', 'FRGI']


In [153]:
#Polygon news request for ticker
url = ('https://api.polygon.io/v2/reference/news?'
      'ticker='+ticker+'&'
      'published_utc/gte='+published+'&'
      'limit='+r_lim+'&'
      'sort=published_utc&'
      'apikey='+polykey
      )

In [154]:
response = requests.get(url)
if response.status_code==200:
    response = response.json()
else:
    print(response.status_code)

In [155]:
print(response['results'][0])

{'id': 'LzRZxvltYXnR-MLwDAegdL7yDZ9zHzKy_cR1kzlKPYI', 'publisher': {'name': 'The Motley Fool', 'homepage_url': 'https://www.fool.com/', 'logo_url': 'https://s3.polygon.io/public/assets/news/logos/themotleyfool.svg', 'favicon_url': 'https://s3.polygon.io/public/assets/news/favicons/themotleyfool.ico'}, 'title': "Here's a Chart Every McDonald's Investor Should See", 'author': 'newsfeedback@fool.com (Demitri Kalogeropoulos)', 'published_utc': '2021-11-12T15:11:00Z', 'article_url': 'https://www.fool.com/investing/2021/11/12/heres-a-chart-every-mcdonalds-investor-should-see/', 'tickers': ['MCD', 'YUM', 'CMG', 'DPZ', 'QSR'], 'amp_url': 'https://www.fool.com/amp/investing/2021/11/12/heres-a-chart-every-mcdonalds-investor-should-see/', 'image_url': 'https://g.foolcdn.com/editorial/images/650334/fast-food-to-go.jpg', 'description': 'The fast-food titan leads the industry with more than just its global reach.', 'keywords': ['investing']}


In [156]:
for i in response['results'][0]:
    print(i)

id
publisher
title
author
published_utc
article_url
tickers
amp_url
image_url
description
keywords


In [157]:
print(response['results'][0]['keywords'])

['investing']


In [158]:
print(response['results'][0]['published_utc'])

2021-11-12T15:11:00Z


In [159]:
print(response['results'][0]['description'])

The fast-food titan leads the industry with more than just its global reach.


In [160]:
for i in response['results']:
    print(i['article_url'])

https://www.fool.com/investing/2021/11/12/heres-a-chart-every-mcdonalds-investor-should-see/
https://www.fool.com/investing/2021/11/09/3-stocks-that-made-a-big-comeback/
https://www.fool.com/investing/2021/11/09/3--things-to-know-before-buying-chipotle-stock/
https://seekingalpha.com/article/4467272-wall-street-breakfast-general-electric-is-splitting-up-podcast-by-pimm-fox
https://www.fool.com/investing/2021/11/09/1-number-you-missed-from-chipotle-q3-earnings/
https://www.fool.com/investing/2021/11/09/1-tasty-reason-for-chipotles-success/
https://seekingalpha.com/article/4467248-wall-street-breakfast-winning-workers
https://www.fool.com/investing/2021/11/06/my-3-favorite-stocks-right-now/
https://www.marketwatch.com/story/a-jog-down-memory-lane-shows-the-factor-that-mattered-most-to-driving-stocks-and-it-sure-wasnt-interest-rates-11636109264
https://www.zacks.com/commentary/1823259/bear-of-the-day-yum-china-yumc
https://www.marketwatch.com/story/roblox-sheds-more-than-1-billion-in-mark

In [ ]:
def query_polygon(ticker, d_start, d_end, pages=1, pageSize=200):
    
    url = ('https://api.polygon.io/v2/reference/news?'
      'ticker='+ticker
      '&published_utc/gte='+d_start
      '&published_utc/lte='+d_end
      '&limit='+pageSize
      '&sort=published_utc'
      '&apikey='+polykey)
    
    response = requests.get(url)
    if response.status_code==200:
        response = response.json()